In [1]:
import gcsfs
import os
import pandas as pd
import netCDF4 as nc
import tempfile
import h3
import numpy as np
from dask.distributed import Client

In [2]:
import os
import pandas as pd

# Specify the directory where your parquet files are stored
parquet_directory = 'data/processed/combined_output/period=1'

# List all parquet files in the directory
parquet_files = [os.path.join(parquet_directory, f) for f in os.listdir(parquet_directory) if f.endswith('.parquet')]

# Read all parquet files and concatenate them into a single DataFrame
df = pd.concat([pd.read_parquet(file) for file in parquet_files])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3636140 entries, (np.int64(612490487321853951), Timestamp('2022-01-01 07:00:00')) to (np.int64(616776967875723263), Timestamp('2022-01-01 20:00:00'))
Data columns (total 1 columns):
 #   Column         Dtype  
---  ------         -----  
 0   precipitation  float64
dtypes: float64(1)
memory usage: 64.5 MB


In [4]:
df.head(10)

precipitation
h3_index           timestamp                         
612490487321853951 2022-01-01 07:00:00       0.000106
612490666202628095 2022-01-01 08:00:00       0.000124
                   2022-01-01 09:00:00       0.000143
612490667936972799 2022-01-01 08:00:00       0.000107
                   2022-01-01 09:00:00       0.000132
612490669539196927 2022-01-01 08:00:00       0.000114
                   2022-01-01 09:00:00       0.000124
612490669816020991 2022-01-01 08:00:00       0.000138
                   2022-01-01 09:00:00       0.000153
612490670094942207 2022-01-01 08:00:00       0.000128